Требования к нереляционным БД

2 - нереляционная БД

2 - красивая структура БД

2 - интерфейс позволяет класть, доставать, удалять данные (проводить операции CRUD - Create, Read, Update, Delete)

2 - два действия помимо CRUD (сортировка, группировка, агрегация, ...)

2 - зависит от БД: Redis - использование ключей, хешей, ...; Neo4j - нахождение путей не только к соседним вершинам, операции с графами; MongoDB, ElasticSearch - работа с текстами или географией; остальные БД - будем договарив

In [4]:
from elasticsearch import Elasticsearch

import json

In [3]:
es = Elasticsearch()

In [ ]:
# Класс, хранящий информацию о статье.
class Article:
    def __init__(self):
        self.url = ""
        self.archive = ""
        self.title = ""
        self.date = ""
        self.text = ""
        self.summary = ""
        
    
    # Конвертация в JSON.
    def toJSON(self):
        res='{"url":"'+self.url+'", "archive":"'+self.archive+'", "title":"'+self.title+'", "date":"'
        res+=self.date+'", "text":"'+self.text+'", "summary":"'+self.summer+'"}'
        return res
    
    # Конвертация в словарь.
    def toDict(self):
        res={"url":self.url, "archive":self.archive, "title":self.title, "date":self.date,\
             "text":self.text, "summary":self.summary}
        return res

In [ ]:
# Класс, хранящий информацию о сгенерированном тексте.
class GeneratedArticle:
    def __init__(self):
        self.text = ""
        self.model = ""
        self.sampling_type = ""
        self.parameter = ""      
        
    
    # Конвертация в JSON.
    def toJSON(self):
        res='{"text":"'+self.text+'", "model":"'+self.model+'", "sampling_type":"'+self.sampling_type+'", "parameter":"'
        res+=self.parameter+'"}'
        return res
    
    # Конвертация в словарь.
    def toDict(self):
        res={"text":self.text, "model":self.model, "sampling_type":self.sampling_type, "parameter":self.parameter}
        return res

# Data exploration

In [12]:
with open('test-stats.jsonl', 'r') as json_file:
        json_list = list(json_file)[:10]
        
# data = []
# for json_str in json_list:
#         result = json.loads(json_str)
#         data.append(clean_data(result['text']))

In [13]:
json_list[0][:40]

'{"url":"http:\\/\\/www.nydailynews.com\\/ar'

In [16]:
for json_str in json_list:
    result = json.loads(json_str)
    print(result.keys())
    print(result['url'])
    print('===============================')
    print(result['archive'])
    print('===============================')
    print(result['title'])
    print('===============================')
    print(result['date'])
    print('===============================')
    print(result['text'])
    print('===============================')
    print(result['summary'])
    print('===============================')
#     print(result['compression'])
#     print('===============================')
#     print(result['coverage'])
#     print('===============================')
#     print(result['density'])
#     print('===============================')
#     print(result['compression_bin'])
#     print('===============================')
#     print(result['coverage_bin'])
#     print('===============================')
#     print(result['density_bin'])
#     print('===============================')
    break
#     data.append(clean_data(result['text']))

dict_keys(['url', 'archive', 'title', 'date', 'text', 'summary', 'compression', 'coverage', 'density', 'compression_bin', 'coverage_bin', 'density_bin'])
http://www.nydailynews.com/archives/news/1995/10/14/1995-10-14_selena_s_last_cries___shot_s.html
http://web.archive.org/web/20090428161725id_/http://www.nydailynews.com:80/archives/news/1995/10/14/1995-10-14_selena_s_last_cries___shot_s.html
SELENA'S LAST CRIES SHOT SINGER BEGGED HELP, NAMED SUSPECT
20090428161725
By MATT SCHWARTZ in Houston and WENDELL JAMIESON in New York Daily News Writers

Saturday, October 14th 1995, 4:22AM

Bleeding from a massive chest wound, Tejano star Selena cried, "Help me! Help me! I've been shot!" and then named her killer with her dying breath.

Shaken witnesses yesterday told a spellbound Houston courtroom how the blood-covered, mortally wounded 23-year-old Hispanic singing sensation burst into the lobby of the Corpus Christi Days Inn last March 31.

Gasping for breath, Selena told motel workers that Yo